# Challenges 3: Stored Procedures and Functions

### Load SQL extension

In [1]:
%load_ext sql

### Establish connection
Store your local credentials in `~/.my.cnf`.

In [2]:
%sql mysql+pymysql:///northwind?unix_socket=/var/run/mysql/mysql.sock&read_default_file=~/.my.cnf

## Goal: Automatic restocking orders

The idea is to evaluate the current stock, including already placed but not yet shipped orders and to decide for which suppliers orders should be issued.

- orders are only placed for suppliers for which at least one product is running out-of-stock
- for suppliers for which at least one product needs to be restocked, all products that are going below reorder-level are stocked up to the reorder-level
- products that are marked as discontinued must be skipped

> General approach: As most procedures in this context (evaluation instead of business operations) result in intermediate tabular data, I am using temporary tables to return results. In general, stored SQL views would be more appropriate but the task of this chapter is to learn stored procedures and functions, so I will use those!

### Procedure: Collect all quantities that have been ordered but not shipped yet

In [59]:
%%sql
CREATE OR REPLACE PROCEDURE preordered_product_quantities()
BEGIN
  DROP TEMPORARY TABLE IF EXISTS preordered_product_quantities;
  CREATE TEMPORARY TABLE preordered_product_quantities
  AS
    SELECT ProductID, SUM(Quantity) AS PreorderedQuantity
    FROM orders
    JOIN order_details USING(OrderID)
    WHERE ShippedDate IS NULL
    GROUP BY ProductID;
END;

 * mysql+pymysql:///northwind?read_default_file=~%2F.my.cnf&unix_socket=%2Fvar%2Frun%2Fmysql%2Fmysql.sock
0 rows affected.


[]

### Procedure: Determine which products are running below reorder-level

In [136]:
%%sql
CREATE OR REPLACE PROCEDURE reorder_amounts()
BEGIN
  CALL preordered_product_quantities();
  DROP TEMPORARY TABLE IF EXISTS reorder_amounts;
  CREATE TEMPORARY TABLE reorder_amounts
  AS
    SELECT
      ProductID,
      UnitsInStock + UnitsOnOrder - PreorderedQuantity AS AvailableStock,
      GREATEST(0, ReorderLevel - (UnitsInStock + UnitsOnOrder - PreorderedQuantity)) AS ReorderAmount
    FROM products
    JOIN preordered_product_quantities USING(ProductID)
    WHERE Discontinued = 'n';
END;


 * mysql+pymysql:///northwind?read_default_file=~%2F.my.cnf&unix_socket=%2Fvar%2Frun%2Fmysql%2Fmysql.sock
0 rows affected.


[]

### Key business logic procedure: Determine which suppliers to place orders for
... and collect all products for these suppliers which are running low, so they can be ordered along in order to save freight costs even if they are not running out yet.

In [137]:
%%sql
CREATE OR REPLACE PROCEDURE prepare_supplier_orders()
BEGIN
  CALL reorder_amounts();
  DROP TEMPORARY TABLE IF EXISTS supplier_orders;
  CREATE TEMPORARY TABLE supplier_orders AS
  WITH
  reorder_suppliers AS ( -- those suppliers for which at least one product is out of stock
    SELECT DISTINCT SupplierID
      FROM reorder_amounts
      JOIN products USING(ProductID)
      WHERE ReorderAmount >= ReorderLevel AND ReorderAmount > 0
  )
  -- restock all items for these suppliers up to reorder level
  SELECT SupplierID, ProductID, ReorderAmount
  FROM reorder_amounts
  JOIN products USING(ProductID)
  JOIN reorder_suppliers USING(SupplierID)
  WHERE ReorderAmount > 0
  ORDER BY SupplierID, ProductID;
END;


 * mysql+pymysql:///northwind?read_default_file=~%2F.my.cnf&unix_socket=%2Fvar%2Frun%2Fmysql%2Fmysql.sock
0 rows affected.


[]

#### Example output of the supplier orders

In [138]:
%%sql
CALL prepare_supplier_orders();
SELECT * FROM supplier_orders;

 * mysql+pymysql:///northwind?read_default_file=~%2F.my.cnf&unix_socket=%2Fvar%2Frun%2Fmysql%2Fmysql.sock
106 rows affected.
12 rows affected.


SupplierID,ProductID,ReorderAmount
1,1,11
1,2,30
3,7,11
6,13,25
7,16,27
7,70,17
12,64,60
12,77,13
16,35,19
20,43,34


## Generate order documents
### First step: Format orders in JSON for passing it to the document generator

#### Function: Supplier contact data as JSON

In [122]:
%%sql

CREATE OR REPLACE FUNCTION get_supplier_json(id INT)
RETURNS JSON
DETERMINISTIC
BEGIN
  RETURN (
    SELECT
      JSON_OBJECT(
        "name", CompanyName,
        "contact", JSON_OBJECT(
          "name", ContactName,
          "title", ContactTitle
        ),
        "address", Address,
        "city", City,
        "region", NULLIF(Region, ''),
        "postal_code", PostalCode,
        "country", Country
      ) AS supplier_json
    FROM suppliers
    WHERE SupplierID = id
  );
END;


 * mysql+pymysql:///northwind?read_default_file=~%2F.my.cnf&unix_socket=%2Fvar%2Frun%2Fmysql%2Fmysql.sock
0 rows affected.


[]

#### Procedure: Collect all supplier contact information as JSON array

In [178]:
%%sql
CREATE OR REPLACE PROCEDURE prepare_supplier_json()
BEGIN
  DROP TEMPORARY TABLE IF EXISTS supplier_json;
  CREATE TEMPORARY TABLE supplier_json
  AS SELECT
    SupplierID,
    get_supplier_json(SupplierID) AS supplier_json
  FROM (SELECT DISTINCT SupplierID FROM supplier_orders) s;
END;


 * mysql+pymysql:///northwind?read_default_file=~%2F.my.cnf&unix_socket=%2Fvar%2Frun%2Fmysql%2Fmysql.sock
0 rows affected.


[]

##### Example output

In [179]:
%%sql
CALL prepare_supplier_orders();
CALL prepare_supplier_json();
SELECT * FROM supplier_json;

 * mysql+pymysql:///northwind?read_default_file=~%2F.my.cnf&unix_socket=%2Fvar%2Frun%2Fmysql%2Fmysql.sock
106 rows affected.
9 rows affected.
9 rows affected.


SupplierID,supplier_json
1,"{""name"": ""Exotic Liquids"", ""contact"": {""name"": ""Charlotte Cooper"", ""title"": ""Purchasing Manager""}, ""address"": ""49 Gilbert St."", ""city"": ""London"", ""region"": null, ""postal_code"": ""EC1 4SD"", ""country"": ""UK""}"
3,"{""name"": ""Grandma Kelly's Homestead"", ""contact"": {""name"": ""Regina Murphy"", ""title"": ""Sales Representative""}, ""address"": ""707 Oxford Rd."", ""city"": ""Ann Arbor"", ""region"": ""MI"", ""postal_code"": ""48104"", ""country"": ""USA""}"
6,"{""name"": ""Mayumi's"", ""contact"": {""name"": ""Mayumi Ohno"", ""title"": ""Marketing Representative""}, ""address"": ""92 Setsuko\r\nChuo-ku"", ""city"": ""Osaka"", ""region"": null, ""postal_code"": ""545"", ""country"": ""Japan""}"
7,"{""name"": ""Pavlova, Ltd."", ""contact"": {""name"": ""Ian Devling"", ""title"": ""Marketing Manager""}, ""address"": ""74 Rose St.\r\nMoonie Ponds"", ""city"": ""Melbourne"", ""region"": ""Victoria"", ""postal_code"": ""3058"", ""country"": ""Australia""}"
12,"{""name"": ""Plutzer Lebensmittelgroßmärkte AG"", ""contact"": {""name"": ""Martin Bein"", ""title"": ""International Marketing Mgr.""}, ""address"": ""Bogenallee 51"", ""city"": ""Frankfurt"", ""region"": null, ""postal_code"": ""60439"", ""country"": ""Germany""}"
16,"{""name"": ""Bigfoot Breweries"", ""contact"": {""name"": ""Cheryl Saylor"", ""title"": ""Regional Account Rep.""}, ""address"": ""3400 - 8th Avenue\r\nSuite 210"", ""city"": ""Bend"", ""region"": ""OR"", ""postal_code"": ""97101"", ""country"": ""USA""}"
20,"{""name"": ""Leka Trading"", ""contact"": {""name"": ""Chandra Leka"", ""title"": ""Owner""}, ""address"": ""471 Serangoon Loop, Suite #402"", ""city"": ""Singapore"", ""region"": null, ""postal_code"": ""0512"", ""country"": ""Singapore""}"
24,"{""name"": ""G'day, Mate"", ""contact"": {""name"": ""Wendy Mackenzie"", ""title"": ""Sales Representative""}, ""address"": ""170 Prince Edward Parade\r\nHunter's Hill"", ""city"": ""Sydney"", ""region"": ""NSW"", ""postal_code"": ""2042"", ""country"": ""Australia""}"
28,"{""name"": ""Gai pâturage"", ""contact"": {""name"": ""Eliane Noz"", ""title"": ""Sales Representative""}, ""address"": ""Bat. B\r\n3, rue des Alpes"", ""city"": ""Annecy"", ""region"": null, ""postal_code"": ""74000"", ""country"": ""France""}"


#### Function: Product order line as JSON

In [140]:
%%sql
CREATE OR REPLACE FUNCTION get_product_order_json(product_id INT, quantity INT)
RETURNS JSON
DETERMINISTIC
BEGIN
  RETURN (
    SELECT
      JSON_OBJECT(
        'name', ProductName,
        'qpu', QuantityPerUnit,
        'units', quantity) AS product_json
    FROM products
    WHERE ProductID = product_id
  );
END;


 * mysql+pymysql:///northwind?read_default_file=~%2F.my.cnf&unix_socket=%2Fvar%2Frun%2Fmysql%2Fmysql.sock
0 rows affected.


[]

#### Procedure: Collect all product order lines per supplier as JSON

In [204]:
%%sql
CREATE OR REPLACE PROCEDURE prepare_product_order_json()
BEGIN
  DROP TEMPORARY TABLE IF EXISTS product_order_json;
  CREATE TEMPORARY TABLE product_order_json (SupplierID INT, product_order_json JSON)
  AS SELECT
    so.SupplierID,
    JSON_ARRAYAGG(
      JSON_EXTRACT(get_product_order_json(ProductID, ReorderAmount), '$')
      ORDER BY ProductName
    ) AS product_order_json
  FROM supplier_orders so
  JOIN products USING(ProductID)
  GROUP BY SupplierID;
END;


 * mysql+pymysql:///northwind?read_default_file=~%2F.my.cnf&unix_socket=%2Fvar%2Frun%2Fmysql%2Fmysql.sock
0 rows affected.


[]

##### Example output

In [205]:
%%sql
CALL prepare_supplier_orders();
CALL prepare_product_order_json();
SELECT * FROM product_order_json;

 * mysql+pymysql:///northwind?read_default_file=~%2F.my.cnf&unix_socket=%2Fvar%2Frun%2Fmysql%2Fmysql.sock
106 rows affected.
9 rows affected.
9 rows affected.


SupplierID,product_order_json
1,"[{""name"": ""Chai"", ""qpu"": ""10 boxes x 20 bags"", ""units"": ""11""},{""name"": ""Chang"", ""qpu"": ""24 - 12 oz bottles"", ""units"": ""30""}]"
3,"[{""name"": ""Uncle Bob's Organic Dried Pears"", ""qpu"": ""12 - 1 lb pkgs."", ""units"": ""11""}]"
6,"[{""name"": ""Konbu"", ""qpu"": ""2 kg box"", ""units"": ""25""}]"
7,"[{""name"": ""Outback Lager"", ""qpu"": ""24 - 355 ml bottles"", ""units"": ""17""},{""name"": ""Pavlova"", ""qpu"": ""32 - 500 g boxes"", ""units"": ""27""}]"
12,"[{""name"": ""Original Frankfurter grüne Soße"", ""qpu"": ""12 boxes"", ""units"": ""13""},{""name"": ""Wimmers gute Semmelknödel"", ""qpu"": ""20 bags x 4 pieces"", ""units"": ""60""}]"
16,"[{""name"": ""Steeleye Stout"", ""qpu"": ""24 - 12 oz bottles"", ""units"": ""19""}]"
20,"[{""name"": ""Ipoh Coffee"", ""qpu"": ""16 - 500 g tins"", ""units"": ""34""}]"
24,"[{""name"": ""Manjimup Dried Apples"", ""qpu"": ""50 - 300 g pkgs."", ""units"": ""14""}]"
28,"[{""name"": ""Camembert Pierrot"", ""qpu"": ""15 - 300 g rounds"", ""units"": ""54""}]"


#### Procedure: Combine contact data and order data for each supplier as JSON

In [209]:
%%sql
CREATE OR REPLACE PROCEDURE prepare_all_supplier_orders()
BEGIN
  CALL prepare_supplier_orders();
  CALL prepare_supplier_json();
  CALL prepare_product_order_json();
  SELECT
    JSON_ARRAYAGG(
      JSON_OBJECT(
        'supplier', JSON_EXTRACT(supplier_json, '$'),
        'products', JSON_EXTRACT(product_order_json, '$')
      )
    ) AS supplier_orders
  FROM (SELECT DISTINCT SupplierID FROM supplier_orders) s
  JOIN supplier_json USING(SupplierID)
  JOIN product_order_json USING(SupplierID);
END;

 * mysql+pymysql:///northwind?read_default_file=~%2F.my.cnf&unix_socket=%2Fvar%2Frun%2Fmysql%2Fmysql.sock
0 rows affected.


[]

#### Pass JSON data on to the Python document generator layer

In [210]:
%%sql supplier_orders <<
CALL prepare_all_supplier_orders();

 * mysql+pymysql:///northwind?read_default_file=~%2F.my.cnf&unix_socket=%2Fvar%2Frun%2Fmysql%2Fmysql.sock
1 rows affected.
Returning data to local variable supplier_orders


### Second step: Use f-string based template to format supplier orders

In [254]:
from IPython.display import display, Markdown, Latex
import json
Markdown(
  '\n\n----\n'.join(
    [
f'''
> __Northwind Traders__\\
1234 NW Market Street\\
Seattle, WA 98107\\
USA

&nbsp;\\
&nbsp;\\
&nbsp;\\
__{ order['supplier']['name'] }__\\
{ order['supplier']['address'] }\\
{ order['supplier']['city'] } { order['supplier']['region'] } { order['supplier']['postal_code'] }\\
{ order['supplier']['country'] }

# Order

Dear { order['supplier']['contact']['name'] },

We would like to place an order for the following items:

| Units | Product Name | Quantity per Unit |
|------:|:-------------|:------------------|
{
  chr(13).join([
    f"| {product['units']} | {product['name']} | {product['qpu']} |"
    for product in order['products']
  ])
}

Please ship to our warehouse at your earliest convenience.

Sincerely,

Pumbaa\\
Purchase Manager\\
Northwind Traders\\
&nbsp;\\
&nbsp;\\
&nbsp;
'''
      for order in json.loads(supplier_orders[0][0])
    ]
  )
)


> __Northwind Traders__\
1234 NW Market Street\
Seattle, WA 98107\
USA

&nbsp;\
&nbsp;\
&nbsp;\
__Exotic Liquids__\
49 Gilbert St.\
London None EC1 4SD\
UK

# Order

Dear Charlotte Cooper,

We would like to place an order for the following items:

| Units | Product Name | Quantity per Unit |
|------:|:-------------|:------------------|
| 11 | Chai | 10 boxes x 20 bags || 30 | Chang | 24 - 12 oz bottles |

Please ship to our warehouse at your earliest convenience.

Sincerely,

Pumbaa\
Purchase Manager\
Northwind Traders\
&nbsp;\
&nbsp;\
&nbsp;


----

> __Northwind Traders__\
1234 NW Market Street\
Seattle, WA 98107\
USA

&nbsp;\
&nbsp;\
&nbsp;\
__Grandma Kelly's Homestead__\
707 Oxford Rd.\
Ann Arbor MI 48104\
USA

# Order

Dear Regina Murphy,

We would like to place an order for the following items:

| Units | Product Name | Quantity per Unit |
|------:|:-------------|:------------------|
| 11 | Uncle Bob's Organic Dried Pears | 12 - 1 lb pkgs. |

Please ship to our warehouse at your earliest convenience.

Sincerely,

Pumbaa\
Purchase Manager\
Northwind Traders\
&nbsp;\
&nbsp;\
&nbsp;


----

> __Northwind Traders__\
1234 NW Market Street\
Seattle, WA 98107\
USA

&nbsp;\
&nbsp;\
&nbsp;\
__Mayumi's__\
92 Setsuko
Chuo-ku\
Osaka None 545\
Japan

# Order

Dear Mayumi Ohno,

We would like to place an order for the following items:

| Units | Product Name | Quantity per Unit |
|------:|:-------------|:------------------|
| 25 | Konbu | 2 kg box |

Please ship to our warehouse at your earliest convenience.

Sincerely,

Pumbaa\
Purchase Manager\
Northwind Traders\
&nbsp;\
&nbsp;\
&nbsp;


----

> __Northwind Traders__\
1234 NW Market Street\
Seattle, WA 98107\
USA

&nbsp;\
&nbsp;\
&nbsp;\
__Pavlova, Ltd.__\
74 Rose St.
Moonie Ponds\
Melbourne Victoria 3058\
Australia

# Order

Dear Ian Devling,

We would like to place an order for the following items:

| Units | Product Name | Quantity per Unit |
|------:|:-------------|:------------------|
| 17 | Outback Lager | 24 - 355 ml bottles || 27 | Pavlova | 32 - 500 g boxes |

Please ship to our warehouse at your earliest convenience.

Sincerely,

Pumbaa\
Purchase Manager\
Northwind Traders\
&nbsp;\
&nbsp;\
&nbsp;


----

> __Northwind Traders__\
1234 NW Market Street\
Seattle, WA 98107\
USA

&nbsp;\
&nbsp;\
&nbsp;\
__Plutzer Lebensmittelgroßmärkte AG__\
Bogenallee 51\
Frankfurt None 60439\
Germany

# Order

Dear Martin Bein,

We would like to place an order for the following items:

| Units | Product Name | Quantity per Unit |
|------:|:-------------|:------------------|
| 13 | Original Frankfurter grüne Soße | 12 boxes || 60 | Wimmers gute Semmelknödel | 20 bags x 4 pieces |

Please ship to our warehouse at your earliest convenience.

Sincerely,

Pumbaa\
Purchase Manager\
Northwind Traders\
&nbsp;\
&nbsp;\
&nbsp;


----

> __Northwind Traders__\
1234 NW Market Street\
Seattle, WA 98107\
USA

&nbsp;\
&nbsp;\
&nbsp;\
__Bigfoot Breweries__\
3400 - 8th Avenue
Suite 210\
Bend OR 97101\
USA

# Order

Dear Cheryl Saylor,

We would like to place an order for the following items:

| Units | Product Name | Quantity per Unit |
|------:|:-------------|:------------------|
| 19 | Steeleye Stout | 24 - 12 oz bottles |

Please ship to our warehouse at your earliest convenience.

Sincerely,

Pumbaa\
Purchase Manager\
Northwind Traders\
&nbsp;\
&nbsp;\
&nbsp;


----

> __Northwind Traders__\
1234 NW Market Street\
Seattle, WA 98107\
USA

&nbsp;\
&nbsp;\
&nbsp;\
__Leka Trading__\
471 Serangoon Loop, Suite #402\
Singapore None 0512\
Singapore

# Order

Dear Chandra Leka,

We would like to place an order for the following items:

| Units | Product Name | Quantity per Unit |
|------:|:-------------|:------------------|
| 34 | Ipoh Coffee | 16 - 500 g tins |

Please ship to our warehouse at your earliest convenience.

Sincerely,

Pumbaa\
Purchase Manager\
Northwind Traders\
&nbsp;\
&nbsp;\
&nbsp;


----

> __Northwind Traders__\
1234 NW Market Street\
Seattle, WA 98107\
USA

&nbsp;\
&nbsp;\
&nbsp;\
__G'day, Mate__\
170 Prince Edward Parade
Hunter's Hill\
Sydney NSW 2042\
Australia

# Order

Dear Wendy Mackenzie,

We would like to place an order for the following items:

| Units | Product Name | Quantity per Unit |
|------:|:-------------|:------------------|
| 14 | Manjimup Dried Apples | 50 - 300 g pkgs. |

Please ship to our warehouse at your earliest convenience.

Sincerely,

Pumbaa\
Purchase Manager\
Northwind Traders\
&nbsp;\
&nbsp;\
&nbsp;


----

> __Northwind Traders__\
1234 NW Market Street\
Seattle, WA 98107\
USA

&nbsp;\
&nbsp;\
&nbsp;\
__Gai pâturage__\
Bat. B
3, rue des Alpes\
Annecy None 74000\
France

# Order

Dear Eliane Noz,

We would like to place an order for the following items:

| Units | Product Name | Quantity per Unit |
|------:|:-------------|:------------------|
| 54 | Camembert Pierrot | 15 - 300 g rounds |

Please ship to our warehouse at your earliest convenience.

Sincerely,

Pumbaa\
Purchase Manager\
Northwind Traders\
&nbsp;\
&nbsp;\
&nbsp;


## Summary
Using Jupyter notebooks with the `ipython-sql` extension is a useful method to prototype SQL database logic and allows to integrate post-procession right away :-)